<a href="https://colab.research.google.com/github/CSC-Artificial-Intelligence/Application-of-Neural-Networks/blob/main/nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
# import numpy as np
# import matplotlib.pyplot as plt

In [3]:
# Define the transform with data augmentation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [4]:
# Load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
class_names = [
    "airplane",										
    "automobile",										
    "bird",										
    "cat",										
    "deer",										
    "dog",										
    "frog",										
    "horse",										
    "ship",										
    "truck",
]

In [6]:


# Define a simple CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 12, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 24, 5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))
        x = self.pool(nn.ReLU()(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = nn.ReLU()( self.fc1(x))
        x = nn.ReLU()( self.fc2(x))        
        x = self.fc3(x)
        return x


In [7]:
model = CNN()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [ ]:
num_epochs = 50

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for i, data in enumerate(train_loader):
        images, labels = data
        optimizer.zero_grad()  # Clear gradients
        outputs = model(images)  # Forward pass
        loss = loss_function(outputs, labels)
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%')

Epoch [1/50], Loss: 2.2299, Train Accuracy: 16.89%
Epoch [2/50], Loss: 1.7867, Train Accuracy: 34.35%
Epoch [3/50], Loss: 1.5430, Train Accuracy: 43.59%
Epoch [4/50], Loss: 1.4241, Train Accuracy: 48.52%
Epoch [5/50], Loss: 1.3248, Train Accuracy: 52.38%
Epoch [6/50], Loss: 1.2375, Train Accuracy: 55.80%
Epoch [7/50], Loss: 1.1687, Train Accuracy: 58.44%
Epoch [8/50], Loss: 1.1096, Train Accuracy: 60.61%
Epoch [9/50], Loss: 1.0594, Train Accuracy: 62.68%
Epoch [10/50], Loss: 1.0117, Train Accuracy: 64.22%
Epoch [11/50], Loss: 0.9665, Train Accuracy: 65.95%
Epoch [12/50], Loss: 0.9261, Train Accuracy: 67.48%
Epoch [13/50], Loss: 0.8874, Train Accuracy: 68.75%
Epoch [14/50], Loss: 0.8526, Train Accuracy: 69.76%
Epoch [15/50], Loss: 0.8190, Train Accuracy: 71.14%
Epoch [16/50], Loss: 0.7904, Train Accuracy: 72.09%
Epoch [17/50], Loss: 0.7555, Train Accuracy: 73.57%
Epoch [18/50], Loss: 0.7335, Train Accuracy: 74.20%
Epoch [19/50], Loss: 0.7018, Train Accuracy: 75.25%
Epoch [20/50], Loss: 

In [1]:
torch.save(model.state_dict(), "trained_model.pth")

NameError: name 'torch' is not defined

In [ ]:
model = CNN()
model.load_state_dict(torch.load("trained_model.pth"))


In [ ]:

# Evaluate on test set
model.eval()  # Set model to evaluation mode
correct = 0
total = 0


with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')